# Description

This kernel will attempt to tackle the prediction of the scalar coupling constant by combining a few ideas I picked up from notebooks and other kernels.

Roughly we will do the following:
* Train models for the molecular properties per 'type' of atom bond and add the predictions as features to the test set (as molecular properties are only given for the training set).
* Train a model per 'type' of atom bond using a combination of distance features, the molecular property features mentioned above and standard interaction forces between atoms (i.e. potentials and so on).


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

from xgboost import XGBRegressor
import lightgbm as lgb

import gc

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
# Evaluation - see https://www.kaggle.com/uberkinder/efficient-metric
def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (pd.Series(y_true)-pd.Series(y_pred)).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()


# Reusable function that does cross-validation and returns a tuple of 
# training scores, validation scores over the folds and predictions on
# the test dataframe
def do_cross_validation(
    train_df
    , y
    , test_df
    , n_folds = 3
    , n_reshuffles = 1
    , shuffle = True
    , random_state=42
    , estimator = 'lightgbm'
    , metric = 'group_mean_log_mae'
    , columns = []
    , cat_columns = []
    , **kwargs
):
    
    train_df_processed = train_df[columns]
    test_df_processed = test_df[columns]

    for c in cat_columns:
        le = LabelEncoder()
        train_df_processed[c] = le.fit_transform(train_df_processed[c])
        test_df_processed[c] = le.transform(test_df_processed[c])
        train_df_processed[c] = train_df_processed[c].astype('category')
        test_df_processed[c] = test_df_processed[c].astype('category')
           

    predictions = pd.DataFrame(data=np.zeros([test_df_processed.shape[0],n_folds*n_reshuffles]),columns=['pred_'+str(s) for s in range(n_folds*n_reshuffles)],index=test_df_processed.index)
    train_score = np.zeros([n_folds*n_reshuffles])
    val_score = np.zeros([n_folds*n_reshuffles]) 
    
    for s in range(n_reshuffles):
        print('Reshuffle: ', s)
        kf = KFold(n_splits=n_folds, shuffle=shuffle, random_state=random_state)
        
        for fold, (train_index, val_index) in enumerate(kf.split(np.array(train_df_processed))):
            print('Training fold: ', fold)

            X_train, X_val = np.array(train_df_processed)[train_index], np.array(train_df_processed)[val_index]
            y_train, y_val = np.array(y)[train_index], np.array(y)[val_index]
            
            if metric == 'group_mean_log_mae':
                X_train_types, X_val_types = np.array(train_df['type'])[train_index], np.array(train_df['type'])[val_index]

            if estimator == 'lightgbm':
                
                lightgbm_params = kwargs['lightgbm_params']
                model = lgb.LGBMRegressor(
                            **lightgbm_params
                            , nthread=-1)

                model.fit(X_train,y_train, 
                              eval_set=[(X_train, y_train),(X_val, y_val)], 
                              eval_metric=lightgbm_params['metric'],
                              verbose=lightgbm_params['verbose'])

            y_train_pred = model.predict(X_train)
            y_val_pred = model.predict(X_val)
            y_pred = model.predict(test_df_processed)
            
            if metric == 'group_mean_log_mae':
                train_score[s*n_folds + fold] = group_mean_log_mae(y_train_pred, y_train, X_train_types)
                val_score[s*n_folds + fold] = group_mean_log_mae(y_val_pred, y_val, X_val_types)
            
            predictions['pred_'+str(s*n_folds + fold)] = y_pred
            
            print("Training score: ", train_score[s*n_folds + fold])
            print("Validiation score: ", val_score[s*n_folds + fold])
            
    
    return train_score, val_score, predictions

# Reusable function that does cross-validation and returns a tuple of 
# training scores, validation scores over the folds and predictions on
# the test dataframe
# def do_cross_validation_efficient(
#     train_df
#     , test_df
#     , label
#     , n_folds = 3
#     , n_reshuffles = 1
#     , shuffle = True
#     , random_state=42
#     , estimator = 'lightgbm'
#     , metric = 'group_mean_log_mae'
#     , columns = []
#     , cat_columns = []
#     , **kwargs
# ):
    
#     train_df_processed = train_df[columns]
#     test_df_processed = test_df[columns]

#     predictions = pd.DataFrame(data=np.zeros([test_df_processed.shape[0],n_folds*n_reshuffles]),columns=['pred_'+str(s) for s in range(n_folds*n_reshuffles)],index=test_df_processed.index)
#     train_score = np.zeros([n_folds*n_reshuffles])
#     val_score = np.zeros([n_folds*n_reshuffles]) 
    
#     for c in cat_columns:
#         le = LabelEncoder()
#         train_df_processed[c] = le.fit_transform(train_df_processed[c])
#         test_df_processed[c] = le.transform(test_df_processed[c])

    
#     train_data = lgb.Dataset(train_df_processed, label=label, feature_name=columns, categorical_feature=cat_columns)
#     result_dict = lgb.cv(kwargs['lightgbm_params'], train_data, shuffle=True, nfold=n_folds, stratified=False)
#     print(result_dict)
    

# Getting the data

In [19]:
train_df = pd.read_csv('champs-scalar-coupling/train.csv')
test_df = pd.read_csv('champs-scalar-coupling/test.csv')
structures_df = pd.read_csv('champs-scalar-coupling/structures.csv')

dipole_moments = pd.read_csv('champs-scalar-coupling/dipole_moments.csv')
magnetic_shielding_tensors = pd.read_csv('champs-scalar-coupling/magnetic_shielding_tensors.csv')
mulliken_charges = pd.read_csv('champs-scalar-coupling/mulliken_charges.csv')
potential_energy = pd.read_csv('champs-scalar-coupling/potential_energy.csv')
scalar_coupling_contributions = pd.read_csv('champs-scalar-coupling/scalar_coupling_contributions.csv')

print("Shape of training df: ",train_df.shape)
print("Shape of testing df: ",test_df.shape)
print("Shape of structures df: ",structures_df.shape)

print("Shape of dipole moments: ",dipole_moments.shape)
print("Shape of magnetic shielding tensors: ",magnetic_shielding_tensors.shape)
print("Shape of mulliken charges: ",mulliken_charges.shape)
print("Shape of potential energy: ",potential_energy.shape)
print("Shape of scalar coupling contributions: ",scalar_coupling_contributions.shape)

Shape of training df:  (4658147, 6)
Shape of testing df:  (2505542, 5)
Shape of structures df:  (2358657, 6)
Shape of dipole moments:  (85003, 4)
Shape of magnetic shielding tensors:  (1533537, 11)
Shape of mulliken charges:  (1533537, 3)
Shape of potential energy:  (85003, 2)
Shape of scalar coupling contributions:  (4658147, 8)


### Joining structure data on train/test set

In [20]:
tmp_with_atom0_info = (train_df
                           .merge( structures_df, left_on = ['molecule_name','atom_index_0'], right_on = ['molecule_name','atom_index'], how = 'left' )
                           .drop('atom_index', axis=1)
                       )
tmp_with_atom0_info = tmp_with_atom0_info.rename(columns={ 'atom' : 'atom_0', 'x' : 'x_0', 'y' : 'y_0', 'z' : 'z_0'})

tmp_with_atom1_info = (tmp_with_atom0_info
                           .merge( structures_df, left_on = ['molecule_name','atom_index_1'], right_on = ['molecule_name','atom_index'], how = 'left' )
                           .drop('atom_index', axis=1)
                       )
tmp_with_atom1_info = tmp_with_atom1_info.rename(columns={ 'atom' : 'atom_1', 'x' : 'x_1', 'y' : 'y_1', 'z' : 'z_1'})

train_df = tmp_with_atom1_info

tmp_with_atom0_info = (test_df
                           .merge( structures_df, left_on = ['molecule_name','atom_index_0'], right_on = ['molecule_name','atom_index'], how = 'left' )
                           .drop('atom_index', axis=1)
                       )
tmp_with_atom0_info = tmp_with_atom0_info.rename(columns={ 'atom' : 'atom_0', 'x' : 'x_0', 'y' : 'y_0', 'z' : 'z_0'})

tmp_with_atom1_info = (tmp_with_atom0_info
                           .merge( structures_df, left_on = ['molecule_name','atom_index_1'], right_on = ['molecule_name','atom_index'], how = 'left' )
                           .drop('atom_index', axis=1)
                       )
tmp_with_atom1_info = tmp_with_atom1_info.rename(columns={ 'atom' : 'atom_1', 'x' : 'x_1', 'y' : 'y_1', 'z' : 'z_1'})

test_df = tmp_with_atom1_info

train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [21]:
del structures_df

### Joining atomic/molecular properties to training data

In [22]:
dipole_moments.head()

,molecule_name,X,Y,Z
0,dsgdb9nsd_000001,0.0000,0.0,0.0000
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511
3,dsgdb9nsd_000005,0.0000,0.0,-2.8937
4,dsgdb9nsd_000007,0.0000,0.0,0.0000


In [23]:
train_df = train_df.merge(dipole_moments, left_on=['molecule_name'], right_on=['molecule_name'])
train_df = train_df.rename(columns={'X' : 'dipole_x', 'Y' : 'dipole_y', 'Z' : 'dipole_z'})
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dipole_x,dipole_y,dipole_z
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.0,0.0,0.0
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.0,0.0,0.0
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.0,0.0,0.0
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.0,0.0,0.0
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.0,0.0,0.0


In [24]:
magnetic_shielding_tensors.head()

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
0,dsgdb9nsd_000001,0,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
1,dsgdb9nsd_000001,1,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861
2,dsgdb9nsd_000001,2,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967
3,dsgdb9nsd_000001,3,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579
4,dsgdb9nsd_000001,4,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013


In [25]:
train_df = train_df.merge(magnetic_shielding_tensors, left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index']).drop('atom_index', axis=1)
train_df = train_df.rename(columns={
        'XX' : 'mag_xx_0'
        , 'YX' : 'mag_yx_0'
        , 'ZX' : 'mag_zx_0'
        , 'XY' : 'mag_xy_0'
        , 'YY' : 'mag_yy_0'
        , 'ZY' : 'mag_zy_0'
        , 'XZ' : 'mag_xz_0'
        , 'YZ' : 'mag_yz_0'
        , 'ZZ' : 'mag_zz_0'
})

train_df = train_df.merge(magnetic_shielding_tensors, left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index']).drop('atom_index', axis=1)
train_df = train_df.rename(columns={
        'XX' : 'mag_xx_1'
        , 'YX' : 'mag_yx_1'
        , 'ZX' : 'mag_zx_1'
        , 'XY' : 'mag_xy_1'
        , 'YY' : 'mag_yy_1'
        , 'ZY' : 'mag_zy_1'
        , 'XZ' : 'mag_xz_1'
        , 'YZ' : 'mag_yz_1'
        , 'ZZ' : 'mag_zz_1'
})
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,mag_zz_0,mag_xx_1,mag_yx_1,mag_zx_1,mag_xy_1,mag_yy_1,mag_zy_1,mag_xz_1,mag_yz_1,mag_zz_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,...,34.0861,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,...,33.8967,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,H,-0.540815,1.447527,-0.876644,...,28.9579,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,H,-0.523814,1.437933,0.906397,...,28.9013,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,...,34.0861,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967


In [26]:
mulliken_charges.head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [27]:
train_df = train_df.merge(mulliken_charges, left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index']).drop('atom_index', axis=1)
train_df = train_df.rename(columns={
        'mulliken_charge' : 'mulliken_charge_0'
})

train_df = train_df.merge(mulliken_charges, left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index']).drop('atom_index', axis=1)
train_df = train_df.rename(columns={
        'mulliken_charge' : 'mulliken_charge_1'
})
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,mag_yx_1,mag_zx_1,mag_xy_1,mag_yy_1,mag_zy_1,mag_xz_1,mag_yz_1,mag_zz_1,mulliken_charge_0,mulliken_charge_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,...,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,0.133921,-0.535689
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,...,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,0.133922,-0.535689
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,H,-0.540815,1.447527,-0.876644,...,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,0.133923,-0.535689
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,H,-0.523814,1.437933,0.906397,...,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,0.133923,-0.535689
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,...,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967,0.133921,0.133922


In [28]:
potential_energy.head()

,molecule_name,potential_energy
0,dsgdb9nsd_000001,-40.52368
1,dsgdb9nsd_000002,-56.56025
2,dsgdb9nsd_000003,-76.42608
3,dsgdb9nsd_000005,-93.42849
4,dsgdb9nsd_000007,-79.83869


In [29]:
train_df = train_df.merge(potential_energy, left_on=['molecule_name'], right_on=['molecule_name'])

train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,mag_zx_1,mag_xy_1,mag_yy_1,mag_zy_1,mag_xz_1,mag_yz_1,mag_zz_1,mulliken_charge_0,mulliken_charge_1,potential_energy
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,...,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,0.133921,-0.535689,-40.52368
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,...,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,0.133922,-0.535689,-40.52368
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,H,-0.540815,1.447527,-0.876644,...,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,0.133923,-0.535689,-40.52368
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,H,-0.523814,1.437933,0.906397,...,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,0.133923,-0.535689,-40.52368
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,...,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967,0.133921,0.133922,-40.52368


In [30]:
# scalar_coupling_contributions.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [31]:
# train_df = train_df.merge(scalar_coupling_contributions, left_on=['molecule_name','atom_index_0','atom_index_1','type'], right_on=['molecule_name','atom_index_0','atom_index_1','type'])

# train_df.head()


,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,mag_xz_1,mag_yz_1,mag_zz_1,mulliken_charge_0,mulliken_charge_1,potential_energy,fc,sd,pso,dso
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,...,-0.0001,0.0007,195.3170,0.133921,-0.535689,-40.52368,83.0224,0.254579,1.25862,0.272010
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,...,-0.0001,0.0007,195.3170,0.133922,-0.535689,-40.52368,83.0222,0.254585,1.25861,0.272013
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,H,-0.540815,1.447527,-0.876644,...,-0.0001,0.0007,195.3170,0.133923,-0.535689,-40.52368,83.0241,0.254634,1.25856,0.272012
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,H,-0.523814,1.437933,0.906397,...,-0.0001,0.0007,195.3170,0.133923,-0.535689,-40.52368,83.0243,0.254628,1.25856,0.272012
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,...,-4.1476,-1.6036,33.8967,0.133921,0.133922,-40.52368,-11.0347,0.352978,2.85839,-3.433600


In [32]:
del potential_energy, mulliken_charges, magnetic_shielding_tensors, dipole_moments, scalar_coupling_contributions

### Adding distance data

In [33]:
train_p_0 = train_df[['x_0', 'y_0', 'z_0']].values
train_p_1 = train_df[['x_1', 'y_1', 'z_1']].values
test_p_0 = test_df[['x_0', 'y_0', 'z_0']].values
test_p_1 = test_df[['x_1', 'y_1', 'z_1']].values

train_df['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)
test_df['dist'] = np.linalg.norm(test_p_0 - test_p_1, axis=1)
train_df['dist_x'] = (train_df['x_0'] - train_df['x_1']) ** 2
test_df['dist_x'] = (test_df['x_0'] - test_df['x_1']) ** 2
train_df['dist_y'] = (train_df['y_0'] - train_df['y_1']) ** 2
test_df['dist_y'] = (test_df['y_0'] - test_df['y_1']) ** 2
train_df['dist_z'] = (train_df['z_0'] - train_df['z_1']) ** 2
test_df['dist_z'] = (test_df['z_0'] - test_df['z_1']) ** 2

train_df['type_0'] = train_df['type'].apply(lambda x: x[0])
test_df['type_0'] = test_df['type'].apply(lambda x: x[0])

train_df['type_1'] = train_df['type'].apply(lambda x: x[1:])
test_df['type_1'] = test_df['type'].apply(lambda x: x[1:])

# Some more distances related to average molecule and type distances
# Freely adapted after https://www.kaggle.com/artgor/brute-force-feature-engineering
def create_features(df):
    df['molecule_couples'] = df.groupby('molecule_name')['id'].transform('count')
    df['molecule_dist_mean'] = df.groupby('molecule_name')['dist'].transform('mean')
    df['molecule_dist_min'] = df.groupby('molecule_name')['dist'].transform('min')
    df['molecule_dist_max'] = df.groupby('molecule_name')['dist'].transform('max')
    df['atom_0_couples_count'] = df.groupby(['molecule_name', 'atom_index_0'])['id'].transform('count')
    df['atom_1_couples_count'] = df.groupby(['molecule_name', 'atom_index_1'])['id'].transform('count')
    
    df[f'molecule_atom_index_0_x_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['x_1'].transform('std')
    df[f'molecule_atom_index_0_y_1_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('mean')
    df[f'molecule_atom_index_0_y_1_mean_diff'] = df[f'molecule_atom_index_0_y_1_mean'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_mean_div'] = df[f'molecule_atom_index_0_y_1_mean'] / df['y_1']
    df[f'molecule_atom_index_0_y_1_max'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('max')
    df[f'molecule_atom_index_0_y_1_max_diff'] = df[f'molecule_atom_index_0_y_1_max'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('std')
    df[f'molecule_atom_index_0_z_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['z_1'].transform('std')
    df[f'molecule_atom_index_0_dist_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('mean')
    df[f'molecule_atom_index_0_dist_mean_diff'] = df[f'molecule_atom_index_0_dist_mean'] - df['dist']
    df[f'molecule_atom_index_0_dist_mean_div'] = df[f'molecule_atom_index_0_dist_mean'] / df['dist']
    df[f'molecule_atom_index_0_dist_max'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('max')
    df[f'molecule_atom_index_0_dist_max_diff'] = df[f'molecule_atom_index_0_dist_max'] - df['dist']
    df[f'molecule_atom_index_0_dist_max_div'] = df[f'molecule_atom_index_0_dist_max'] / df['dist']
    df[f'molecule_atom_index_0_dist_min'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('min')
    df[f'molecule_atom_index_0_dist_min_diff'] = df[f'molecule_atom_index_0_dist_min'] - df['dist']
    df[f'molecule_atom_index_0_dist_min_div'] = df[f'molecule_atom_index_0_dist_min'] / df['dist']
    df[f'molecule_atom_index_0_dist_std'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('std')
    df[f'molecule_atom_index_0_dist_std_diff'] = df[f'molecule_atom_index_0_dist_std'] - df['dist']
    df[f'molecule_atom_index_0_dist_std_div'] = df[f'molecule_atom_index_0_dist_std'] / df['dist']
    df[f'molecule_atom_index_1_dist_mean'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('mean')
    df[f'molecule_atom_index_1_dist_mean_diff'] = df[f'molecule_atom_index_1_dist_mean'] - df['dist']
    df[f'molecule_atom_index_1_dist_mean_div'] = df[f'molecule_atom_index_1_dist_mean'] / df['dist']
    df[f'molecule_atom_index_1_dist_max'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('max')
    df[f'molecule_atom_index_1_dist_max_diff'] = df[f'molecule_atom_index_1_dist_max'] - df['dist']
    df[f'molecule_atom_index_1_dist_max_div'] = df[f'molecule_atom_index_1_dist_max'] / df['dist']
    df[f'molecule_atom_index_1_dist_min'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('min')
    df[f'molecule_atom_index_1_dist_min_diff'] = df[f'molecule_atom_index_1_dist_min'] - df['dist']
    df[f'molecule_atom_index_1_dist_min_div'] = df[f'molecule_atom_index_1_dist_min'] / df['dist']
    df[f'molecule_atom_index_1_dist_std'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('std')
    df[f'molecule_atom_index_1_dist_std_diff'] = df[f'molecule_atom_index_1_dist_std'] - df['dist']
    df[f'molecule_atom_index_1_dist_std_div'] = df[f'molecule_atom_index_1_dist_std'] / df['dist']
    df[f'molecule_atom_1_dist_mean'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('mean')
    df[f'molecule_atom_1_dist_min'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('min')
    df[f'molecule_atom_1_dist_min_diff'] = df[f'molecule_atom_1_dist_min'] - df['dist']
    df[f'molecule_atom_1_dist_min_div'] = df[f'molecule_atom_1_dist_min'] / df['dist']
    df[f'molecule_atom_1_dist_std'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('std')
    df[f'molecule_atom_1_dist_std_diff'] = df[f'molecule_atom_1_dist_std'] - df['dist']
    df[f'molecule_type_0_dist_std'] = df.groupby(['molecule_name', 'type_0'])['dist'].transform('std')
    df[f'molecule_type_0_dist_std_diff'] = df[f'molecule_type_0_dist_std'] - df['dist']
    df[f'molecule_type_1_dist_std'] = df.groupby(['molecule_name', 'type_1'])['dist'].transform('std')
    df[f'molecule_type_1_dist_std_diff'] = df[f'molecule_type_1_dist_std'] - df['dist']
    df[f'molecule_type_dist_mean'] = df.groupby(['molecule_name', 'type'])['dist'].transform('mean')
    df[f'molecule_type_dist_mean_diff'] = df[f'molecule_type_dist_mean'] - df['dist']
    df[f'molecule_type_dist_mean_div'] = df[f'molecule_type_dist_mean'] / df['dist']
    df[f'molecule_type_dist_max'] = df.groupby(['molecule_name', 'type'])['dist'].transform('max')
    df[f'molecule_type_dist_min'] = df.groupby(['molecule_name', 'type'])['dist'].transform('min')
    df[f'molecule_type_dist_std'] = df.groupby(['molecule_name', 'type'])['dist'].transform('std')
    df[f'molecule_type_dist_std_diff'] = df[f'molecule_type_dist_std'] - df['dist']

    return df

train_types = train_df['type']
test_types = test_df['type']

#train_df = create_features(train_df)
#test_df = create_features(test_df)

In [34]:
# Some of the features with 'std' are not valid, so imputing with 0
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

# Predict missing data on test set

In [38]:
# lightgbm_params = {'num_leaves': 50,
#           'min_child_samples': 79,
#           'min_data_in_leaf': 100,
#           'objective': 'regression',
#           'max_depth': 9,
#           'learning_rate': 0.2,
#           "boosting_type": "gbdt",
#           "subsample_freq": 1,
#           "subsample": 0.9,
#           "bagging_seed": 11,
#           "metric": 'mae',
#           "verbosity": -1,
#           'reg_alpha': 0.1,
#           'reg_lambda': 0.3,
#           'colsample_bytree': 1.0,
                   
#           'early_stopping_rounds':200,
#           'n_estimators':25000,
#           'verbose':1000,
#           'num_threads':11
                   
#          }

In [36]:
# features_to_predict_missing_test_data = [
#      'type',
#        'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1',
#        'x_1', 'y_1', 'z_1', 'dist',
#        'dist_x', 'dist_y', 'dist_z', 'type_0', 'type_1'    
# ]

# features_to_predict = ['dipole_x', 'dipole_y', 'dipole_z', 'mag_xx_0',
#        'mag_yx_0', 'mag_zx_0', 'mag_xy_0', 'mag_yy_0', 'mag_zy_0', 'mag_xz_0',
#        'mag_yz_0', 'mag_zz_0', 'mag_xx_1', 'mag_yx_1', 'mag_zx_1', 'mag_xy_1',
#        'mag_yy_1', 'mag_zy_1', 'mag_xz_1', 'mag_yz_1', 'mag_zz_1',
#        'mulliken_charge_0', 'mulliken_charge_1', 'potential_energy']

# cat_columns = ['type','type_0','type_1','atom_0','atom_1']

In [ ]:
# missing_features_prediction = pd.DataFrame(data=None,columns=features_to_predict,index=test_df.index)

# for f in features_to_predict:
#     print('Feature: ',f)
#     train_score, val_score, predictions = do_cross_validation(
#             train_df[features_to_predict_missing_test_data]
#             ,train_df[f]
#             ,test_df[features_to_predict_missing_test_data]
#             ,lightgbm_params=lightgbm_params
#             ,columns = features_to_predict_missing_test_data
#             ,cat_columns = cat_columns)
    
#     missing_features_prediction[f] = predictions.mean(axis=1)
    
#     gc.collect()

In [ ]:
missing_features_prediction.to_pickle("missing_feature_predictions.pkl")

In [ ]:
missing_features_prediction.head()

In [ ]:
train_df.to_pickle("train_df.pkl")
train_df.head()

In [ ]:
test_df.to_pickle("test_df.pkl")
test_df.head()

In [ ]:
train_df.dtypes.value_counts()

In [ ]:
test_df.dtypes.value_counts()

### TODO
## Final model, all features + predicted missing features, per atom binding type

In [3]:
train_df = pd.read_pickle("train_df.pkl")
test_df = pd.read_pickle("test_df.pkl")
missing_feature_prediction_df = pd.read_pickle("missing_feature_prediction.pkl")

In [4]:
missing_feature_prediction_df['id'] = test_df['id']

In [5]:
test_df = test_df.merge(missing_feature_prediction_df,on='id')
test_df.shape

(2505542, 98)

In [6]:
del missing_feature_prediction_df
gc.collect()

35

In [7]:
train_df.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type',
       'scalar_coupling_constant', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1',
       'x_1', 'y_1', 'z_1', 'dipole_x', 'dipole_y', 'dipole_z', 'mag_xx_0',
       'mag_yx_0', 'mag_zx_0', 'mag_xy_0', 'mag_yy_0', 'mag_zy_0', 'mag_xz_0',
       'mag_yz_0', 'mag_zz_0', 'mag_xx_1', 'mag_yx_1', 'mag_zx_1', 'mag_xy_1',
       'mag_yy_1', 'mag_zy_1', 'mag_xz_1', 'mag_yz_1', 'mag_zz_1',
       'mulliken_charge_0', 'mulliken_charge_1', 'potential_energy', 'dist',
       'dist_x', 'dist_y', 'dist_z', 'type_0', 'type_1', 'molecule_couples',
       'molecule_dist_mean', 'molecule_dist_min', 'molecule_dist_max',
       'atom_0_couples_count', 'atom_1_couples_count',
       'molecule_atom_index_0_x_1_std', 'molecule_atom_index_0_y_1_mean',
       'molecule_atom_index_0_y_1_mean_diff',
       'molecule_atom_index_0_y_1_mean_div', 'molecule_atom_index_0_y_1_max',
       'molecule_atom_index_0_y_1_max_diff', 'molecule_atom_index_0_y_1_std

In [8]:
columns = ['atom_0', 'x_0', 'y_0', 'z_0', 'atom_1',
       'x_1', 'y_1', 'z_1', 'dipole_x', 'dipole_y', 'dipole_z', 'mag_xx_0',
       'mag_yx_0', 'mag_zx_0', 'mag_xy_0', 'mag_yy_0', 'mag_zy_0', 'mag_xz_0',
       'mag_yz_0', 'mag_zz_0', 'mag_xx_1', 'mag_yx_1', 'mag_zx_1', 'mag_xy_1',
       'mag_yy_1', 'mag_zy_1', 'mag_xz_1', 'mag_yz_1', 'mag_zz_1',
       'mulliken_charge_0', 'mulliken_charge_1', 'potential_energy', 'dist',
       'dist_x', 'dist_y', 'dist_z', 'molecule_couples',
       'molecule_dist_mean', 'molecule_dist_min', 'molecule_dist_max',
       'atom_0_couples_count', 'atom_1_couples_count',
       'molecule_atom_index_0_x_1_std', 'molecule_atom_index_0_y_1_mean',
       'molecule_atom_index_0_y_1_mean_diff',
       'molecule_atom_index_0_y_1_mean_div', 'molecule_atom_index_0_y_1_max',
       'molecule_atom_index_0_y_1_max_diff', 'molecule_atom_index_0_y_1_std',
       'molecule_atom_index_0_z_1_std', 'molecule_atom_index_0_dist_mean',
       'molecule_atom_index_0_dist_mean_diff',
       'molecule_atom_index_0_dist_mean_div', 'molecule_atom_index_0_dist_max',
       'molecule_atom_index_0_dist_max_diff',
       'molecule_atom_index_0_dist_max_div', 'molecule_atom_index_0_dist_min',
       'molecule_atom_index_0_dist_min_diff',
       'molecule_atom_index_0_dist_min_div', 'molecule_atom_index_0_dist_std',
       'molecule_atom_index_0_dist_std_diff',
       'molecule_atom_index_0_dist_std_div', 'molecule_atom_index_1_dist_mean',
       'molecule_atom_index_1_dist_mean_diff',
       'molecule_atom_index_1_dist_mean_div', 'molecule_atom_index_1_dist_max',
       'molecule_atom_index_1_dist_max_diff',
       'molecule_atom_index_1_dist_max_div', 'molecule_atom_index_1_dist_min',
       'molecule_atom_index_1_dist_min_diff',
       'molecule_atom_index_1_dist_min_div', 'molecule_atom_index_1_dist_std',
       'molecule_atom_index_1_dist_std_diff',
       'molecule_atom_index_1_dist_std_div', 'molecule_atom_1_dist_mean',
       'molecule_atom_1_dist_min', 'molecule_atom_1_dist_min_diff',
       'molecule_atom_1_dist_min_div', 'molecule_atom_1_dist_std',
       'molecule_atom_1_dist_std_diff', 'molecule_type_0_dist_std',
       'molecule_type_0_dist_std_diff', 'molecule_type_1_dist_std',
       'molecule_type_1_dist_std_diff', 'molecule_type_dist_mean',
       'molecule_type_dist_mean_diff', 'molecule_type_dist_mean_div',
       'molecule_type_dist_max', 'molecule_type_dist_min',
       'molecule_type_dist_std', 'molecule_type_dist_std_diff']

cat_columns = ['atom_0','atom_1']

lightgbm_params = {'num_leaves': 50,
          'min_child_samples': 79,
          'min_data_in_leaf': 100,
          'objective': 'regression',
          'max_depth': 9,
          'learning_rate': 0.2,
          "boosting_type": "gbdt",
          "subsample_freq": 1,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'reg_alpha': 0.1,
          'reg_lambda': 0.3,
          'colsample_bytree': 1.0,
                   
          'early_stopping_rounds':200,
          'n_estimators':20000,
          'verbose':1000
         }

In [9]:
train_df.type.value_counts()

3JHC    1510379
2JHC    1140674
1JHC     709416
3JHH     590611
2JHH     378036
3JHN     166415
2JHN     119253
1JHN      43363
Name: type, dtype: int64

In [10]:
gc.collect()

0

In [11]:
print(train_df.info())
for col in train_df.columns:
    if train_df[col].dtype == 'float64':
        train_df[col] = train_df[col].astype(np.float32)
print(train_df.info())

print(test_df.info())
for col in test_df.columns:
    if test_df[col].dtype == 'float64':
        test_df[col] = test_df[col].astype(np.float32)
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4658147 entries, 0 to 4658146
Data columns (total 99 columns):
id                                      int64
molecule_name                           object
atom_index_0                            int64
atom_index_1                            int64
type                                    object
scalar_coupling_constant                float64
atom_0                                  object
x_0                                     float64
y_0                                     float64
z_0                                     float64
atom_1                                  object
x_1                                     float64
y_1                                     float64
z_1                                     float64
dipole_x                                float64
dipole_y                                float64
dipole_z                                float64
mag_xx_0                                float64
mag_yx_0                         

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2505542 entries, 0 to 2505541
Data columns (total 98 columns):
id                                      int64
molecule_name                           object
atom_index_0                            int64
atom_index_1                            int64
type                                    object
atom_0                                  object
x_0                                     float64
y_0                                     float64
z_0                                     float64
atom_1                                  object
x_1                                     float64
y_1                                     float64
z_1                                     float64
dist                                    float64
dist_x                                  float64
dist_y                                  float64
dist_z                                  float64
type_0                                  object
type_1                            

In [12]:
print(test_df.info())
for col in test_df.columns:
    if test_df[col].dtype == 'float64':
        test_df[col] = test_df[col].astype(np.float32)
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2505542 entries, 0 to 2505541
Data columns (total 98 columns):
id                                      int64
molecule_name                           object
atom_index_0                            int64
atom_index_1                            int64
type                                    object
atom_0                                  object
x_0                                     float32
y_0                                     float32
z_0                                     float32
atom_1                                  object
x_1                                     float32
y_1                                     float32
z_1                                     float32
dist                                    float32
dist_x                                  float32
dist_y                                  float32
dist_z                                  float32
type_0                                  object
type_1                            

In [13]:
types = train_df.type.unique()
types = ['3JHC']

total_predictions = pd.Series()

for t in types:
    print('Type: ',t)
    train_score, val_score, predictions = do_cross_validation(
            train_df[train_df.type==t]
            ,train_df[train_df.type==t]['scalar_coupling_constant']
            ,test_df[test_df.type==t]
            ,lightgbm_params=lightgbm_params
            ,columns = columns
            ,cat_columns = cat_columns)
    
    predictions.mean(axis=1).to_csv(f'{t}_predictions.csv')
    
    total_predictions = total_predictions.append(predictions.mean(axis=1))

Type:  3JHC
Reshuffle:  0
Training fold:  0
Training until validation scores don't improve for 200 rounds.
[1000]	training's l1: 0.766821	valid_1's l1: 0.854674
[2000]	training's l1: 0.634923	valid_1's l1: 0.781159
[3000]	training's l1: 0.551078	valid_1's l1: 0.745324
[4000]	training's l1: 0.488774	valid_1's l1: 0.724149
[5000]	training's l1: 0.439077	valid_1's l1: 0.709657
[6000]	training's l1: 0.39788	valid_1's l1: 0.699747
[7000]	training's l1: 0.362215	valid_1's l1: 0.691601
[8000]	training's l1: 0.331461	valid_1's l1: 0.685561
[9000]	training's l1: 0.304031	valid_1's l1: 0.680321
[10000]	training's l1: 0.279729	valid_1's l1: 0.676286
[11000]	training's l1: 0.258153	valid_1's l1: 0.673049
[12000]	training's l1: 0.238637	valid_1's l1: 0.670264
[13000]	training's l1: 0.220929	valid_1's l1: 0.6677
[14000]	training's l1: 0.204952	valid_1's l1: 0.665545
[15000]	training's l1: 0.19033	valid_1's l1: 0.663769
[16000]	training's l1: 0.176932	valid_1's l1: 0.6621
[17000]	training's l1: 0.164

In [30]:
test_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,...,mag_zx_1,mag_xy_1,mag_yy_1,mag_zy_1,mag_xz_1,mag_yz_1,mag_zz_1,mulliken_charge_0,mulliken_charge_1,potential_energy
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,...,-1.141665,-4.841588,163.436218,-2.070283,0.167825,-1.619104,211.348434,0.236130,-0.368003,-326.688629
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,...,-8.434301,24.797152,93.826202,-0.594701,-18.310312,-2.113447,151.421051,0.162334,-0.148747,-336.599609
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,...,-4.882360,-9.747013,61.710735,-3.314898,3.058597,-5.579417,116.464928,0.222256,0.003988,-345.016418
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,...,-12.594582,-0.311534,126.886208,-1.432368,-19.072588,-5.330511,175.096512,0.164841,-0.302636,-413.192383
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,...,0.925977,11.946037,123.292076,0.035085,1.878938,-1.238003,169.322769,0.208852,0.002557,-369.832672


In [26]:
types = train_df.type.unique()
submission_df = pd.concat([
    pd.read_csv(f"{x}_predictions.csv",index_col=0,header=None) for x in types],axis=0).sort_index()

In [37]:
final_df = test_df[['id']]
final_df['scalar_coupling_constant'] = submission_df.loc[:,1]

#test_df[['id']].merge(pd.DataFrame(data=submission_df,index=submission_df.index,columns=['scalar_coupling_constant']),left_index=True,right_index=True)

In [38]:
final_df.head()

,id,scalar_coupling_constant
0,4658147,7.204760
1,4658148,149.319536
2,4658149,3.784811
3,4658150,142.253792
4,4658151,0.107931


In [39]:
file_name = "solution_2.csv"
message = "Model per type - LightGBM"
header = ['id','scalar_coupling_constant']



pd.DataFrame(
    data=list(zip([x for x in final_df['id'].tolist()], [float(x) for x in final_df.scalar_coupling_constant.tolist()]))
).to_csv('{}'.format(file_name), index=False, header=header)

In [88]:
# %%bash -s "$file_name" "$message"
# kaggle competitions submit -c champs-scalar-coupling -f $1 -m "$2"

Process is terminated.
